## <font color='#4285f4'>Overview</font>

This notebook focuses on customer segmentation. It creates JSON views to simplify querying existing segmentation data, and it generates, explores, and visualizes embeddings for the `customer_marketing_profile` table in BigQuery (as defined below), enabling advanced analytics and customer segmentation for Chocolate AI. The notebook shows how you can easily query complex segment data using JSON, and it demonstrates how to find matches for a net new segment using vector search in BigQuery.

```sql
CREATE TABLE IF NOT EXISTS `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
(
    customer_id                            INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_profile_data                  JSON             OPTIONS(description="The raw data we know about a customer."),
    customer_profile_data_embedding        ARRAY<FLOAT64>   OPTIONS(description="Vector embedding of the customer_profile_data column."),
    customer_loyalty_data                  JSON             OPTIONS(description="Data Beans data about the customer's purchases and reviews in JSON format."),
    customer_loyalty_data_embedding        ARRAY<FLOAT64>   OPTIONS(description="Vector embedding of the customer_loyalty_data column."),
    customer_segmentation_data             JSON             OPTIONS(description="The generated customer segmentation data in JSON format."),
    customer_segmentation_data_embedding   ARRAY<FLOAT64>   OPTIONS(description="Vector embedding of the customer_segmentation_data column."),
    customer_marketing_insights            STRING           OPTIONS(description="Generated text summary of customer_profile_data, customer_loyalty_data, and customer_segmentation_data."),
    customer_marketing_insights_embedding  ARRAY<FLOAT64>   OPTIONS(description="Vector embedding of the customer_marketing_insights column.")
)
CLUSTER BY customer_id;
```

Process Flow:
1. Create JSON Views:
    - Creates three views in BigQuery to flatten nested JSON data from the customer_marketing_profile table:
      - customer_marketing_profile_data: Flattens customer_profile_data.
      - customer_marketing_profile_loyalty: Flattens customer_loyalty_data.
      - customer_marketing_profile_segments: Flattens customer_segmentation_data.
      - Creates a combined view customer_360 that joins these flattened views with the main table to provide a unified view of customer information.
2. Explore JSON Segmentation Data:
    - Provides example queries demonstrating how to analyze customer segments using the created views.
    - These queries explore segments based on combinations like "Loyalty Status" and "Generation," analyze demographics of engaged customers on Twitter, identify high-value customers based on spending, and segment customers based on loyalty and average order value.
3. Generate Embeddings:
    - Create Embedding Model: Sets up a connection to Vertex AI and creates a BigQuery remote model (chocolate_ai.google-textembedding) for generating text embeddings.
    - Generate Embeddings: Uses the ML.GENERATE_EMBEDDING function to create embeddings for each customer's customer_profile_data, customer_loyalty_data, customer_segmentation_data, and customer_marketing_insights. It uses different task types for different embedding columns based on the intended use case.
    - Create Vector Index (Optional): Provides instructions for creating a vector index on customer_marketing_insights_embedding to improve the performance of vector search queries.
4. Explore Embeddings:
    - Dynamic Segments with Vector Search: Demonstrates using vector search based on semantic similarity to find customers matching a given description (e.g., "Young professional").
    - Visualize Customer Segments with Clustered Embeddings:
      - Dimensionality Reduction: Applies t-SNE to reduce the dimensionality of the embeddings for visualization purposes.
      - Visualization Function: Defines a function VisualizeEmbeddings to create scatter plots of customer segments based on their reduced-dimension embeddings.
      - Explanation Function: Defines a function ExplainEmbeddings that uses Gemini to explain the key differences between customers in a given segment.
      - Visualization and Explanation: Calls the VisualizeEmbeddings and ExplainEmbeddings functions to visualize and explain segments for selected categories like 'loyalty_status'.

Author: Paul Ramsey

## <font color='#4285f4'>License</font>

```
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################
```

## <font color='#4285f4'>Initialize</font>

Update the values of the variables below to match your environment.

In [ ]:
# Update these variables to match your environment
location="us-central1" # Your region
bigquery_location = "${bigquery_location}" # Must be "us" or "eu"

# Define vertex ai connection name
vertex_ai_connection_name = "vertex-ai" # Can be any valid name.

### Do not change the values in this cell below this line ###
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")
print(f"location = {location}")
print(f"bigquery_location = {bigquery_location}")
print(f"vertex_ai_connection_name = {vertex_ai_connection_name}")

In [ ]:
import google.auth
import requests
import json
import os
import random
import datetime
import time
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception
import logging
from IPython.display import HTML
import markdown

import re
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_20newsgroups
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from scipy.spatial.distance import cdist

from google.cloud import bigquery
client = bigquery.Client()

## <font color='#4285f4'>Helper Functions</font>

Run the cells below to initialize helper functions which are utilized throughout the notebook.

#### RetryCondition()

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### GeminiLLM()

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### PrettyPrintJson()

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

#### RunQuery()

In [ ]:
def RunQuery(sql, job_config = None):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    if job_config == None:
      job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

#### DisplayMarkdown()

In [ ]:
def DisplayMarkdown(text):
  """
  Displays text in markdown/HTML format in a Colab notebook.

  Args:
    text: The text to display. Can be plain text or Markdown.
  """

  formatted_text = markdown.markdown(text)  # Convert to HTML if necessary
  display(HTML(formatted_text))

#### createVertexAIConnection()

In [ ]:
# Function to create Vertex AI connection
def createVertexAIConnection(params):
  """Creates a Vertex AI connection."""

  # First find the connection
  # https://cloud.google.com/bigquery/docs/reference/bigqueryconnection/rest/v1/projects.locations.connections/list
  project_id = params["project_id"]
  bigquery_location = params["bigquery_location"]
  vertex_ai_connection_name = params["vertex_ai_connection_name"]
  url = f"https://bigqueryconnection.googleapis.com/v1/projects/{project_id}/locations/{bigquery_location}/connections"

  # Gather existing connections
  json_result = restAPIHelper(url, "GET", None)
  print(f"createVertexAIConnection (GET) json_result: {json_result}")

  # Test to see if connection exists, if so return
  if "connections" in json_result:
    for item in json_result["connections"]:
      print(f"BigLake Connection: {item['name']}")
      # NOTE: We cannot test the complete name since it contains the project number and not id
      if item["name"].endswith(f"/locations/{bigquery_location}/connections/{vertex_ai_connection_name}"):
        print("Connection already exists")
        serviceAccountId = item["cloudResource"]["serviceAccountId"]
        return serviceAccountId

  # Create the connection
  # https://cloud.google.com/bigquery/docs/reference/bigqueryconnection/rest/v1/projects.locations.connections/create
  print("Creating Vertex AI Connection")

  url = f"https://bigqueryconnection.googleapis.com/v1/projects/{project_id}/locations/{bigquery_location}/connections?connectionId={vertex_ai_connection_name}"

  request_body = {
      "friendlyName": "notebook_connection",
      "description": "Vertex AI Colab Notebooks Connection for Data Analytics Golden Demo",
      "cloudResource": {}
  }

  json_result = restAPIHelper(url, "POST", request_body)

  serviceAccountId = json_result["cloudResource"]["serviceAccountId"]
  print("Vertex AI Connection created: ", serviceAccountId)
  return serviceAccountId

#### setProjectLevelIamPolicy()

In [ ]:
# Helper function for IAM role bindings
def setProjectLevelIamPolicy(params, accountWithPrefix, role):
  """Sets the Project Level IAM policy."""

  # Get the current bindings (if the account has access then skip)
  # https://cloud.google.com/resource-manager/reference/rest/v1/projects/getIamPolicy
  project_id = params["project_id"]

  url = f"https://cloudresourcemanager.googleapis.com/v1/projects/{project_id}:getIamPolicy"

  request_body = { }
  json_result = restAPIHelper(url, "POST", request_body)
  print(f"setProjectLevelIamPolicy (GET) json_result: {json_result}")

  # Test to see if permissions exist
  if "bindings" in json_result:
    for item in json_result["bindings"]:
      if item["role"] == role:
        members = item["members"]
        for member in members:
          if member == accountWithPrefix:
            print("Permissions exist")
            return

  # Take the existing bindings and we need to append the new permission
  # Otherwise we loose the existing permissions
  if "bindings" in json_result:
    bindings = json_result["bindings"]
  else:
    bindings = []

  new_permission = {
      "role": role,
      "members": [ accountWithPrefix ]
      }

  bindings.append(new_permission)

  # https://cloud.google.com/resource-manager/reference/rest/v1/projects/setIamPolicy
  url = f"https://cloudresourcemanager.googleapis.com/v1/projects/{project_id}:setIamPolicy"

  request_body = { "policy" : {
      "bindings" : bindings
      }
  }

  print(f"Permission bindings: {bindings}")

  json_result = restAPIHelper(url, "POST", request_body)
  print()
  print(f"json_result: {json_result}")
  print()
  print(f"Project Level IAM Permissions set for {accountWithPrefix} {role}")

#### restAPIHelper()

In [ ]:
# Rest API Helper Function
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)


## <font color='#4285f4'>Create JSON Views</font>

In the notebook `Synthetic-Data-Generation-Customers-Marketing-Profile`, we used functions to generate customer profile and segment data as JSON. Since we leveraged Gemini's [Controlled Generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output#model_behavior_and_response_schema) feature, we can rely on a consistent schema in the JSON objects, which allows us to create views so that we can more easily query the JSON data.

### Create profile data view: `customer_marketing_profile_data`

In [ ]:
%%bigquery

CREATE OR REPLACE VIEW `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_data` AS
SELECT
    customer_id,
    JSON_VALUE(customer_profile_data.children) AS children,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.coffee_preferences), ",") AS coffee_preferences,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.content_interaction), ",") AS content_interaction,
    CAST(JSON_VALUE(customer_profile_data.customer_age) AS INT64) AS customer_age,
    JSON_VALUE(customer_profile_data.education) AS education,
    JSON_VALUE(customer_profile_data.facebook_bio) AS facebook_bio,
    JSON_VALUE(customer_profile_data.facebook_engagement) AS facebook_engagement,
    JSON_VALUE(customer_profile_data.facebook_handle) AS facebook_handle,
    JSON_VALUE(customer_profile_data.instagram_bio) AS instagram_bio,
    JSON_VALUE(customer_profile_data.instagram_engagement) AS instagram_engagement,
    JSON_VALUE(customer_profile_data.instagram_handle) AS instagram_handle,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.interests), ",") AS interests,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.lifestyle), ",") AS lifestyle,
    JSON_VALUE(customer_profile_data.linkedin_bio) AS linkedin_bio,
    JSON_VALUE(customer_profile_data.linkedin_engagement) AS linkedin_engagement,
    JSON_VALUE(customer_profile_data.linkedin_handle) AS linkedin_handle,
    JSON_VALUE(customer_profile_data.martial_status) AS martial_status,
    JSON_VALUE(customer_profile_data.occupation) AS occupation,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.solicated_buying_habits), ",") AS solicated_buying_habits,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_profile_data.sports), ",") AS sports,
    JSON_VALUE(customer_profile_data.tiktok_bio) AS tiktok_bio,
    JSON_VALUE(customer_profile_data.tiktok_handle) AS tiktok_handle,
    JSON_VALUE(customer_profile_data.twitter_bio) AS twitter_bio,
    JSON_VALUE(customer_profile_data.twitter_engagement) AS twitter_engagement,
    JSON_VALUE(customer_profile_data.twitter_handle) AS twitter_handle,
    JSON_VALUE(customer_profile_data.youtube_bio) AS youtube_bio,
    JSON_VALUE(customer_profile_data.youtube_handle) AS youtube_handle,
    (
        SELECT STRING_AGG(CONCAT(
            'contact_reason:', JSON_VALUE(interaction, '$.contact_reason'), '; ',
            'resolution_time:', JSON_VALUE(interaction, '$.resolution_time'), '; ',
            'sentiment_analysis:', JSON_VALUE(interaction, '$.sentiment_analysis')
        ), ' | ')
        FROM UNNEST(JSON_QUERY_ARRAY(customer_profile_data, '$.customer_service_interactions')) AS interaction
    ) AS customer_service_interactions
  FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`;

### Create loyalty view: `customer_marketing_profile_loyalty`

In [ ]:
%%bigquery

CREATE OR REPLACE VIEW `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_loyalty` AS
SELECT
    customer_id,
    CAST(JSON_VALUE(customer_loyalty_data.average_amount_spent_per_order) AS BIGNUMERIC) AS average_amount_spent_per_order,
    CAST(JSON_VALUE(customer_loyalty_data.last_order_date) AS TIMESTAMP) AS last_order_date,
    JSON_VALUE(customer_loyalty_data.latest_review_sentiment) AS latest_review_sentiment,
    CAST(JSON_VALUE(customer_loyalty_data.most_frequent_purchase_location) AS INT64) AS most_frequent_purchase_location,
    CAST(JSON_VALUE(customer_loyalty_data.negative_review_percentage) AS NUMERIC) AS negative_review_percentage,
    CAST(JSON_VALUE(customer_loyalty_data.neutral_review_percentage) AS NUMERIC) AS neutral_review_percentage,
    CAST(JSON_VALUE(customer_loyalty_data.positive_review_percentage) AS NUMERIC) AS positive_review_percentage,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_loyalty_data.purchase_locations), ",") AS purchase_locations,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_loyalty_data.top_3_favorite_menu_items), ",") AS top_3_favorite_menu_items,
    CAST(JSON_VALUE(customer_loyalty_data.total_amount_spent) AS BIGNUMERIC) AS total_amount_spent,
    CAST(JSON_VALUE(customer_loyalty_data.total_orders) AS INT64) AS total_orders,
    CAST(JSON_VALUE(customer_loyalty_data.total_reviews) AS INT64) AS total_reviews
  FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`;

### Create segments view: `customer_marketing_profile_segments`

In [ ]:
%%bigquery

CREATE OR REPLACE VIEW `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_segments` AS
SELECT
    customer_id,
    REPLACE(JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Benefits Sought`), " ", "") AS benefits_sought,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Browsing Behavior`) AS browsing_behavior,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Loyalty Status`) AS loyalty_status,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Occasion/Timing`) AS occasion_timing,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Purchase History`) AS purchase_history,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Spending Habits`) AS spending_habits,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`Usage Frequency`) AS usage_frequency,
    JSON_VALUE(customer_segmentation_data.customer_segments.behavioral_segmentation.`User Status`) AS user_status,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`At-Risk Customers`), ",") AS at_risk_customers,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`First-Time Customers`), ",") AS first_time_customers,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`Former Customers`), ",") AS former_customers,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`Inactive Customers`), ",") AS inactive_customers,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`Loyal Advocates`), ",") AS loyal_advocates,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`New Leads`), ",") AS new_leads,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`Potential Customers`), ",") AS potential_customers,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.customer_lifecycle_segmentation.`Repeat Customers`), ",") AS repeat_customers,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Age`) AS age,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Education`) AS education,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Ethnicity`) AS ethnicity,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Family Size`) AS family_size,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Gender`) AS gender,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Generation`) AS generation,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Income`) AS income,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Language`) AS language,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Marital Status`) AS marital_status,
    JSON_VALUE(customer_segmentation_data.customer_segments.demographic_segmentation.`Occupation`) AS occupation,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`City`) AS city,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Climate`) AS climate,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Country`) AS country,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Population Density`) AS population_density,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Region`) AS region,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Time Zone`) AS time_zone,
    JSON_VALUE(customer_segmentation_data.customer_segments.geographic_segmentation.`Urban/Rural`) AS urban_rural,
    JSON_VALUE(customer_segmentation_data.customer_segments.needs_based_segmentation.`Challenges`) AS challenges,
    JSON_VALUE(customer_segmentation_data.customer_segments.needs_based_segmentation.`Goals`) AS goals,
    JSON_VALUE(customer_segmentation_data.customer_segments.needs_based_segmentation.`Pain Points`) AS pain_points,
    JSON_VALUE(customer_segmentation_data.customer_segments.needs_based_segmentation.`Priorities`) AS priorities,
    JSON_VALUE(customer_segmentation_data.customer_segments.needs_based_segmentation.`Specific Needs`) AS specific_needs,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Attitudes`) AS attitudes,
    ARRAY_TO_STRING(JSON_VALUE_ARRAY(customer_segmentation_data.customer_segments.psychographic_segmentation.`Hobbies`), ",") AS hobbies,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Interests`) AS interests,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Lifestyle`) AS lifestyle,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Motivations`) AS motivations,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Personality`) AS personality,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Social Class`) AS social_class,
    JSON_VALUE(customer_segmentation_data.customer_segments.psychographic_segmentation.`Values`) AS customer_values,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Adoption Rate`) AS adoption_rate,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Browsers`) AS browsers,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Devices`) AS devices,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Internet Connectivity`) AS internet_connectivity,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Operating Systems`) AS operating_systems,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Social Media Platforms`) AS social_media_platforms,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Software`) AS software,
    JSON_VALUE(customer_segmentation_data.customer_segments.technographic_segmentation.`Tech Savviness`) AS tech_savviness,
    JSON_VALUE(customer_segmentation_data.customer_segments.value_based_segmentation.`Cost-Benefit Analysis`) AS cost_benefit_analysis,
    JSON_VALUE(customer_segmentation_data.customer_segments.value_based_segmentation.`Perceived Value`) AS perceived_value,
    JSON_VALUE(customer_segmentation_data.customer_segments.value_based_segmentation.`Price Sensitivity`) AS price_sensitivity,
    JSON_VALUE(customer_segmentation_data.customer_segments.value_based_segmentation.`Willingness to Pay`) AS willingness_to_pay
FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`;



### Create combined view: `customer_360`

In [ ]:
%%bigquery

CREATE OR REPLACE VIEW `${project_id}.${bigquery_chocoate_ai_dataset}.customer_360` AS
SELECT
    mp.customer_id,
    -- Customer Marketing Profile Summary
    cmp.customer_marketing_insights,
    -- Customer Marketing Profile Segments
    mp.benefits_sought,
    mp.browsing_behavior,
    mp.loyalty_status,
    mp.occasion_timing,
    mp.purchase_history,
    mp.spending_habits,
    mp.usage_frequency,
    mp.user_status,
    mp.at_risk_customers,
    mp.first_time_customers,
    mp.former_customers,
    mp.inactive_customers,
    mp.loyal_advocates,
    mp.new_leads,
    mp.potential_customers,
    mp.repeat_customers,
    mp.age,
    mp.education,
    mp.ethnicity,
    mp.family_size,
    mp.gender,
    mp.generation,
    mp.income,
    mp.language,
    mp.marital_status,
    mp.occupation,
    mp.city,
    mp.climate,
    mp.country,
    mp.population_density,
    mp.region,
    mp.time_zone,
    mp.urban_rural,
    mp.challenges,
    mp.goals,
    mp.pain_points,
    mp.priorities,
    mp.specific_needs,
    mp.attitudes,
    mp.hobbies,
    mp.interests,
    mp.lifestyle,
    mp.motivations,
    mp.personality,
    mp.social_class,
    mp.customer_values,
    mp.adoption_rate,
    mp.browsers,
    mp.devices,
    mp.internet_connectivity,
    mp.operating_systems,
    mp.social_media_platforms,
    mp.software,
    mp.tech_savviness,
    mp.cost_benefit_analysis,
    mp.perceived_value,
    mp.price_sensitivity,
    mp.willingness_to_pay,
    -- Customer Profile
    cp.children,
    cp.coffee_preferences,
    cp.content_interaction,
    cp.customer_age,
    cp.facebook_bio,
    cp.facebook_engagement,
    cp.facebook_handle,
    cp.instagram_bio,
    cp.instagram_engagement,
    cp.instagram_handle,
    cp.linkedin_bio,
    cp.linkedin_engagement,
    cp.linkedin_handle,
    cp.martial_status,
    cp.solicated_buying_habits,
    cp.sports,
    cp.tiktok_bio,
    cp.tiktok_handle,
    cp.twitter_bio,
    cp.twitter_engagement,
    cp.twitter_handle,
    cp.youtube_bio,
    cp.youtube_handle,
    cp.customer_service_interactions,
    -- Customer Loyalty
    cl.average_amount_spent_per_order,
    cl.last_order_date,
    cl.latest_review_sentiment,
    cl.most_frequent_purchase_location,
    cl.negative_review_percentage,
    cl.neutral_review_percentage,
    cl.positive_review_percentage,
    cl.purchase_locations,
    cl.top_3_favorite_menu_items,
    cl.total_amount_spent,
    cl.total_orders,
    cl.total_reviews
  FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_segments` AS mp
    INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_data` AS cp ON mp.customer_id = cp.customer_id
    INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_loyalty` AS cl ON mp.customer_id = cl.customer_id
    INNER JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile` AS cmp ON mp.customer_id = cmp.customer_id;

## <font color='#4285f4'>Explore JSON Segmentation Data</font>

This section provides an example of how you can use the data generated and derived in this notebook to gain insights into your customers and prepare for new marketing campaigns.

#### Explore customer segments

In [ ]:
%%bigquery

--This query identifies the most common "Loyalty Status" and "Generation"
--combinations among customers. This helps understand which generations we are
--effectively reaching, allowing for more targeted messaging
-- and product development.

SELECT
    loyalty_status,
    generation,
    COUNT(*) AS customer_count
FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_segments`
GROUP BY
    loyalty_status,
    generation
ORDER BY
    customer_count DESC;

#### Explore customer profiles

In [ ]:
%%bigquery

-- This query finds the average customer age for those who actively engage with
-- the brand on Twitter and have mentioned "coffee" in their Twitter bio. This
-- helps understand the demographics of engaged customers on Twitter and tailor
-- content accordingly.

SELECT
    AVG(customer_age) AS average_age
FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_data`
WHERE
    twitter_engagement = 'Active' AND twitter_bio LIKE '%coffee%';

#### Explore loyalty data

In [ ]:
%%bigquery

-- This query calculates the average total amount spent by customers who have
-- placed more than 50 orders and have left at least one review. This helps
-- identify high-value customers and understand their spending patterns, which
-- can inform loyalty programs and targeted promotions.


SELECT
    AVG(total_amount_spent) AS average_spending
FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile_loyalty`
WHERE
    total_orders > 50 AND total_reviews > 0;

#### Explore customer 360 data

In [ ]:
%%bigquery

-- This query segments customers based on their "Loyalty Status" from the
-- customer_marketing_profile_segments view and their "average_amount_spent_per_order"
-- from the customer_loyalty view. This allows you to analyze the relationship
-- between loyalty status and spending habits, enabling you to tailor marketing
-- efforts and loyalty programs to different customer segments.

SELECT
    loyalty_status,
    AVG(average_amount_spent_per_order) AS average_order_value,
    COUNT(*) AS customer_count
FROM
    `${project_id}.${bigquery_chocoate_ai_dataset}.customer_360`
GROUP BY
    loyalty_status
ORDER BY
    customer_count DESC;

## <font color='#4285f4'>Generate embeddings</font>

This section adds vector embeddings for each JSON and STRING column in the `customer_marketing_profile` table, which opens up new capabilities, including semantic search, retrieval augemented generation (RAG), and dynamic customer segmentation.

### Create Embedding Model endpoint

In [ ]:
# NOTE: You can skip this step if you deployed via Terraform as the endpoint was
#       already created for you. This cell is provided for reference to help you
#       follow this pattern in your own environment.

# Define params as dictionary
params = { "project_id" : project_id,
           "bigquery_location" : bigquery_location,
           "user" : user,
           "vertex_ai_connection_name" : vertex_ai_connection_name
           }

# Create the BigQuery External Connection that will be used to call the Vertex AI
# Set the required permissions on the external connection's service principal
vertexAIServiceAccountId = createVertexAIConnection(params)
params["vertexAIServiceAccountId"] = vertexAIServiceAccountId

# To call Vision API
setProjectLevelIamPolicy(params, f"serviceAccount:{vertexAIServiceAccountId}", "roles/serviceusage.serviceUsageConsumer")

# To call GENERATE TEXT
setProjectLevelIamPolicy(params, f"serviceAccount:{vertexAIServiceAccountId}","roles/aiplatform.user")

# Create the text embedding model
# Working:
sql = f"""CREATE MODEL IF NOT EXISTS `${project_id}.${bigquery_chocoate_ai_dataset}.google-textembedding`
REMOTE WITH CONNECTION `{project_id}.{bigquery_location}.{vertex_ai_connection_name}`
OPTIONS (endpoint = 'text-embedding-004');"""

RunQuery(sql)


### Generate embeddings

> NOTE: The demo data set already has embeddings created for you, so you can just read through this step without executing the cells if desired.

Google's latest embedding models now support task types, which increase the accuracy of vector search results based on the nature of the underlying data and your use case.

The new embeddings models support the following task types:
      
|Task type|Embeddings optimization criteria|
|:-|:-|
|`SEMANTIC_SIMILARITY`|Semantic similarity. Use this task type when retrieving similar texts from the corpus.|
|`RETRIEVAL_QUERY`|Document search and information retrieval. Use `RETRIEVAL_QUERY` for query texts, and `RETRIEVAL_DOCUMENT` for documents to be retrieved.|
|`QUESTION_ANSWERING`|Questions and answers applications such as RAG. Use `QUESTION_ANSWERING` for question texts, and `RETRIEVAL_DOCUMENT` for documents to be retrieved.|
|`FACT_VERIFICATION`|Document search for fact verification. Use `FACT_VERIFICATION` for the target text, and `RETRIEVAL_DOCUMENT` for documents to be retrieved.|
|`CODE_RETRIEVAL_QUERY`|Code search. Use `CODE_RETRIEVAL_QUERY` for query text, and `RETRIEVAL_DOCUMENT` for code blocks to be retrieved (available on embedding model `text-embedding-preview-0815` and later)|
|`CLASSIFICATION`|Text classification. Use this task type for training a small classification model with the embedding.|
|`CLUSTERING`|Text clustering. Use this task type for k-means or other clustering analysis.|

For example, if you are building a RAG system for a question and answering use case, you may specify task type `RETRIEVAL_DOCUMENT` for generating embeddings for building with vector search, and specify `QUESTION_ANSWERING` for embeddings for question texts. Thus you should see improved search quality compared to using `SEMANTIC_SIMILARITY` for both query and document. Likewise, you may use `RETRIEVAL_QUERY` for queries for document search, and `FACT_VERIFICATION` for queries for finding documents for fact checking.

Embeddings with task type `CLASSIFICATION` are useful for classifying texts with its semantics for use cases such as customer and product segmentation.

For our use case, we'll use the `SEMANTIC_SIMILARITY` task type for the `customer_marketing_insights_embedding` so that we can search for target customers using natural language, and we'll use the `CLUSTERING` task type for the other three embedding columns so that we can group customers together based on their similarity to each other.

In [ ]:
# Populate customer_loyalty_data_embedding column with embeddings
# Checking for nulls after update and rerunning if nulls are found to work around quota constraints

# Define columns to update and task type to use for embeddings
# Embeddings will be stored in a column named <column name>_embedding
columns_to_embed = [
    ['customer_profile_data','CLUSTERING'],
    ['customer_loyalty_data', 'CLUSTERING'],
    ['customer_segmentation_data','CLUSTERING'],
    ['customer_marketing_insights','SEMANTIC_SIMILARITY']
  ]

for column in columns_to_embed:
  print(f"Populating column {column[0]}_embedding with embeddings using task type {column[1]}")

  sleep_time_seconds = 5

  sql = f"""SELECT COUNT(*) AS null_count FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
        WHERE ARRAY_LENGTH({column[0]}_embedding) = 0"""

  result = RunQuery(sql)
  null_count = result.iloc[0,0]
  limit = null_count

  while null_count > 0:
      last_null_count = null_count

      sql = f"""UPDATE `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile` AS t1
            SET {column[0]}_embedding = t2.ml_generate_embedding_result
            FROM ML.GENERATE_EMBEDDING(
                MODEL `${project_id}.${bigquery_chocoate_ai_dataset}.google-textembedding`,
                (SELECT customer_id, TO_JSON_STRING({column[0]}, false) as content
                FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
                WHERE ARRAY_LENGTH({column[0]}_embedding) = 0
                LIMIT {str(limit)}),
                STRUCT(
                  TRUE AS flatten_json_output,
                  '{column[1]}' as task_type,
                  768 AS output_dimensionality
                  )
            ) AS t2
            WHERE t1.customer_id = t2.customer_id;"""

      result = RunQuery(sql)

      sql = f"""SELECT COUNT(*) AS null_count FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
        WHERE ARRAY_LENGTH({column[0]}_embedding) = 0"""

      result = RunQuery(sql)
      null_count = result.iloc[0,0]

      if null_count > 0:
          print(f"Found null count: {null_count}. Waiting {sleep_time_seconds} seconds before retry.")
          time.sleep(sleep_time_seconds)

          if null_count >= last_null_count:
            # Increase sleep time and decrease limit if we're not making progress
            sleep_time_seconds = sleep_time_seconds * 1.5 if sleep_time_seconds < 30 else 30
            limit = int(limit / 1.5) if limit > 10 else 10

          print(f"Retrying with with limit set to {limit}.")

  print(f"Done populating column {column[0]}_embedding with embeddings.")


### Optional: Create vector index

If you have a table with at least 5000 rows, you can create a vector index to perform approximate nearest neighbor (ANN) search, which is much more performant on large data sets. Tables with less than 5000 rows must use exact nearest neighbor (ENN) search.

This example uses the [IVF algorithm](https://cloud.google.com/bigquery/docs/vector-index#ivf-index). IVF is an inverted file index, which uses a k-means algorithm to cluster the vector data, and then partitions the vector data based on those clusters. When you use the VECTOR_SEARCH function to search the vector data, it can use these partitions to reduce the amount of data it needs to read in order to determine a result.

For large batch search use cases, BigQuery also supports the [TreeAH algorithm](https://cloud.google.com/bigquery/docs/vector-index#tree-ah-index) for vector indexing. TreeAH is a type of vector index that uses Google's ScaNN algorithm. It works as follows:
- The base table is divided into smaller, more manageable shards.
- A clustering model is trained, with the number of clusters derived from the leaf_node_embedding_count option in tree_ah_options.
- The vectors are product quantized and stored in the index tables.
- During VECTOR_SEARCH, a candidate list for each query vector is efficiently computed using asymmetric hashing, which is hardware-optimized for approximate distance calculations. These candidates are then re-scored and re-ranked using exact embeddings.

The TreeAH algorithm is optimized for batch queries that process hundreds or more query vectors.

BigQuery limits you to one vector index per table, so we have chosen to index the `customer_marketing_insights_embedding` column. You could choose to index one of the other columns in this table instead using the syntax below if desired.

```sql
CREATE OR REPLACE VECTOR INDEX customer_segmentation_data_embedding_ivf
ON `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`(customer_segmentation_data_embedding)
OPTIONS (index_type = 'IVF', distance_type = 'COSINE');

CREATE OR REPLACE VECTOR INDEX customer_loyalty_data_embedding_ivf
ON `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`(customer_loyalty_data_embedding)
OPTIONS (index_type = 'IVF', distance_type = 'COSINE');

CREATE OR REPLACE VECTOR INDEX customer_profile_data_embedding_ivf
ON `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`(customer_profile_data_embedding)
OPTIONS (index_type = 'IVF', distance_type = 'COSINE');
```

In [ ]:
%%bigquery

-- Requires minimum of 5000 rows to create ANN index
DROP VECTOR INDEX IF EXISTS customer_marketing_insights_embedding_ivf
ON `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`;

CREATE VECTOR INDEX customer_marketing_insights_embedding_ivf
ON `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`(customer_marketing_insights_embedding)
OPTIONS (index_type = 'IVF', distance_type = 'COSINE');


## <font color='#4285f4'>Explore Embeddings</font>

#### Get Dynamic Segments with Vector Search on Semantic Similarity Embeddings

The query below uses vector search to return a list of the top 5 (top_k) matches based on semantic similarity to the search phrase "Young professional". This provides additional flexibility to build custom segments of customers that don't align with the pre-defined segments in the `customer_segmentation_data` column. You can use a similar query to search based on vector similarity for the other embedding columns we generated earlier as well.

In [ ]:
%%bigquery

-- Ref: https://cloud.google.com/bigquery/docs/vector-search
SELECT
  distance,
  base.customer_id,
  base.customer_marketing_insights,
  base.customer_profile_data,
  base.customer_loyalty_data,
  base.customer_segmentation_data
FROM VECTOR_SEARCH(
  -- base table or subquery
  (
    SELECT * FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
  ),

  -- embedding column to search in base table - must be of type ARRAY<FLOAT64>
  'customer_marketing_insights_embedding',

  -- query table or subquery - this is where you generate the search embedding
  (
    SELECT ml_generate_embedding_result, content AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `${project_id}.${bigquery_chocoate_ai_dataset}.google-textembedding`,
        (
          -- Search term
          SELECT "Young professional" AS content
        ),
        STRUCT(
          TRUE AS flatten_json_output,
          'SEMANTIC_SIMILARITY' as task_type,
          768 AS output_dimensionality
        )
    )
  ),
  top_k => 5,
  distance_type => 'COSINE'
);


### Visualize Customer Segments with Clustered Embeddings

#### Perform Dimensionality Reduction

Our embeddings contain 768 dimensions, which is difficult to visualize. So, we can use the t-Distributed Stochastic Neighbor Embedding (t-SNE) approach to perform dimensionality reduction, enabling us to visualize the embeddings and spot patterns in our customer base.

The t-SNE technique reduces the number of dimensions, while preserving clusters (points that are close together stay close together). For the original data, the model tries to construct a distribution over which other data points are "neighbors" (e.g., they share a similar meaning). It then optimizes an objective function to keep a similar distribution in the visualization.

In [ ]:
# Get embeddings and customer_360 data in a dataframe

sql = """SELECT cmp.customer_id,
    cmp.customer_segmentation_data_embedding AS Embeddings,
    c360.*
  FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile` cmp
  JOIN `${project_id}.${bigquery_chocoate_ai_dataset}.customer_360` c360
    ON c360.customer_id = cmp.customer_id"""

result = RunQuery(sql)
result.head()


In [ ]:
# Show count of current dimensions in embeddings
print(f"Number of dimensions per embedding: {len(result['Embeddings'][0])}")

# Convert result['Embeddings'] Pandas series to a np.array of float32
X = np.array(result['Embeddings'].to_list(), dtype=np.float32)
print(f"Shape of converted np.array: {X.shape}")

# Apply t-SNE
print("Performing dimensionality reduction via t-SNE...")
tsne = TSNE(random_state=0, max_iter=1000)
tsne_results = tsne.fit_transform(X)

# **Create df_tsne with customer_id**
df_tsne = pd.DataFrame({'customer_id': result['customer_id'],
                       'TSNE1': tsne_results[:, 0],
                       'TSNE2': tsne_results[:, 1]})

#### Define Visualization and Explanation Functinos

In [ ]:
# Visualize clustered embeddings
def VisualizeEmbeddings(df, segment):
  fig, ax = plt.subplots(figsize=(8,6))
  sns.set_style('darkgrid', {"grid.color": ".6", "grid.linestyle": ":"})

  merged_df = pd.merge(df_tsne, result[['customer_id', segment]], on='customer_id')

  sns.scatterplot(data=merged_df, x='TSNE1', y='TSNE2', hue=segment, palette='hls')

  plt.title(f'Scatter plot of "{segment}" segment using t-SNE');
  plt.xlabel('TSNE1');
  plt.ylabel('TSNE2');
  plt.axis('equal')

  # Calculate centroids
  centroids = merged_df.groupby(segment).agg({'TSNE1': 'mean', 'TSNE2': 'mean'})

  # Label centroids with nearest customer_id and store for legend
  centroid_labels = {}
  centroid_data = {} # Dictionary to store segment and customer_id pairs
  for segment_name, centroid in centroids.iterrows():
    distances = cdist([centroid], merged_df[merged_df[segment] == segment_name][['TSNE1', 'TSNE2']])
    nearest_index = distances.argmin()
    nearest_customer_id = merged_df[merged_df[segment] == segment_name]['customer_id'].iloc[nearest_index]

    centroid_labels[segment_name] = f"{segment_name} (Cust. {nearest_customer_id})"
    centroid_data[segment_name] = nearest_customer_id  # Store only customer_id

    ax.annotate(nearest_customer_id, (centroid['TSNE1'], centroid['TSNE2']),
                textcoords="offset points", xytext=(5,5), ha='center', fontsize=8)

  # Update legend with centroid labels
  handles, labels = ax.get_legend_handles_labels()
  for i, label in enumerate(labels):
    if label in centroid_labels:
      labels[i] = centroid_labels[label]
  ax.legend(handles, labels, title=segment, loc="upper left", bbox_to_anchor=(1, 1))

  plt.show()
  plt.close()

  return centroid_data

def ExplainEmbeddings(segment_centroid_pair):
  # Define in_string for query
  in_string = ''
  for k, v in segment_centroid_pair.items():
    in_string += f"{v},"
  in_string = f"({in_string[:-1]})"

  # Get raw segment data from BigQuery for centroids
  sql = f"""SELECT customer_id, customer_segmentation_data FROM `${project_id}.${bigquery_chocoate_ai_dataset}.customer_marketing_profile`
    WHERE customer_id IN {in_string}"""
  segment_result = RunQuery(sql)

  # Define prompt
  prompt = f"""The following customers have been selected as representative of a unique customer segment based on a segment category called {segment}:
  {['Customer ' + str(y) + ' is representative of segment "' + str(x) + '"' for x, y in segment_centroid_pair.items()]}

  Here are is the segmentation data for each customer identified above:
  {['Segment data for customer_id ' + str(row['customer_id']) + ': ' + str(row['customer_segmentation_data']) for index, row in segment_result.iterrows()]}

  Explain in summary format 1/ the key differences between each of these customers that is related to segment category "{segment}", 2/ the key differences between each of these customers that is NOT related to segment category "{segment}", and 3/ the key SIMILARITIES between these customers that is NOT related to segment category "{segment}". Think step by step and explain your reasoning.
  """

  response_schema = {
    "type": "object",
    "required": [
      "explanation"
    ],
    "properties": {
      "explanation": {
        "type": "string"
      }
    }
  }

  # Have Gemini explain the segmentation:
  response = GeminiLLM(prompt, response_schema = response_schema)
  return response


#### Visualize Embedding-based Segments

In [ ]:
segments_to_visualize = [
  'age',
  'family_size',
  'gender',
  'generation',
  'marital_status',
  'children',
  'martial_status',
  'loyalty_status',
  'spending_habits',
  'usage_frequency',
]

for segment in segments_to_visualize:
  segment_centroid_pair = VisualizeEmbeddings(df_tsne, segment)

#### Visualize and Explain Embedding-based Segments

In [ ]:
segments_to_visualize_and_explain = [
  'loyalty_status'
]

response = ''
for segment in segments_to_visualize_and_explain:
  segment_centroid_pair = VisualizeEmbeddings(df_tsne, segment)
  response = json.loads(ExplainEmbeddings(segment_centroid_pair))
  DisplayMarkdown(response['explanation'])
